In [1]:
import pickle
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression

In [2]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (150, 50, 50, 3)


In [3]:
# Attention, pour la régression logistique, les images doivent être applaties (flattened) en dimension 1
N = len(noms)

visages = visages.reshape(N, -1)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(visages, noms)

LogisticRegression(max_iter=1000)

In [4]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe
user = {}
test = "\n"
setNom = list(set(noms))
for i in range(len(setNom)):
    user[i] = setNom[i]
    test += str(i) + "." + setNom[i] + "\n"

In [ ]:
numAdmin = list(map(int, input("Qui sont admis : " + test).split()))
admin = []
for num in numAdmin:
    admin.append(user[num])
while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = log_reg.predict(visage_redimensionne)
            print(admin)
            if texte[0] in admin:
                data = texte[0] + " admis"
                col = (255, 0, 0)
            else:
                col = (0, 255, 0)
                data = texte[0] + "non admis"
            
            cv2.putText(trame, data, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), col, 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()

['Quentin']
['Quentin']
['Quentin']
['Quentin']
['Quentin']
